In [ ]:
import sqlite3 as sql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bidi.algorithm import get_display
import arabic_reshaper



In [ ]:
def yesterday(date: str):
    dateList = date.split("/")
    lastMonthLength = 0
    
    if int(dateList[1]) > 7 and int(dateList[1]) <=12:
        lastMonthLength = 30
    elif int(dateList[1]) > 1 and int(dateList[1]) <=7:
        lastMonthLength = 31
    elif int(dateList[1]) == 1:
        lastMonthLength = 29
    
    y = {'year': dateList[0], 'month': dateList[1] , 'day': dateList[2]  }
    
    if int(dateList[2]) > 1:
        y['day'] = int(dateList[2]) - 1
        if y['day'] < 10:
            y['day'] = "0" + str(y['day'])
    elif int(dateList[2]) == 1 and int(dateList[1]) > 1:
        y['month'] = int(dateList[1]) - 1
        if y['month'] < 10:
            y['month'] = "0" + str(y['month'])
        if int(dateList[1]) <= 7:
            y['day'] = 31
        else:
            y['day'] = 30
    elif int(dateList[2]) == 1 and int(dateList[1]) == 1:
        y['year'] = int(dateList[0]) - 1
        y['month'] = 12
        y['day'] = 29
    
    return ("/".join(list(map(str, [y['year'], y['month'], y['day']]))))
          


def nDaysBefore(date: str , n: int):
    listOfDates = []
    for i in range(n):
        if i==0:
            listOfDates.append(yesterday(date))
        else:
            listOfDates.append(yesterday(listOfDates[-1]))
    return(list(reversed(listOfDates)))

        



In [ ]:
df = pd.read_excel('D:\\CRM\\BigData\\Calls_Management\\جدول_تماسها.xlsx' , dtype= 'str')
df= df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
conn = sql.connect('D:\\CRM\\BigData\\Calls_Management\\Calls.db')
df.to_sql('AllData', conn, if_exists = 'replace', index= False)



In [ ]:
dates4SQL = nDaysBefore("1401/06/14" , 30)
df = pd.DataFrame()

for day in dates4SQL:
    querry = '''
                SELECT Company, Date, AllCall, AllCallUniqe, AllCallConnect, AllCallUniqeConnect
                FROM AllData 
                WHERE Date = "{}"                 
                '''.format(day)
    df2 = pd.read_sql(querry, conn)
    df = pd.concat([df, df2], axis=0, ignore_index = True)

df = df.convert_dtypes()
df.dropna(axis=0, thresh= 6, inplace= True)


In [ ]:
#int(df[df['Date']=='1400/12/17'][df['Company']== 'تهران']['AllCall'])
listOfProvinces = ['خراسان جنوبی', 'تهران', 'البرز', 'اصفهان', 'گیلان', 'سمنان',
       'چهار محال و بختیاری', 'کردستان', 'لرستان', 'زنجان',
       'آذربایجان شرقی', 'کرمان', 'یزد', 'هرمزگان', 'مرکزی',
       'آذربایجان غربی', 'خراسان شمالی', 'مشهد', 'خوزستان', 'همدان',
       'شیراز', 'سیستان و بلوچستان', 'کاشان', 'قم', 'مازندران',
       'اردبیل', 'گلستان', 'ایلام', 'قزوین', 'کرمانشاه',
       'کهکیلویه و بویر احمد', 'فارس', 'بوشهر', 'سبزوار', 'خراسان رضوی']

In [ ]:
for c in listOfProvinces:
    y_allCall = []
    y_allCallUnique = []
    y_allCallConnect  = []
    y_allCallUniqeConnect = []
    
    for day in dates4SQL:
        try:
            y_allCall.append(int(df[df['Date']== day][df['Company']== c]['AllCall']))
        except TypeError:
            y_allCall.append(0)
        try:
            y_allCallUnique.append(int(df[df['Date']== day][df['Company']== c]['AllCallUniqe']))
        except TypeError:
            y_allCallUnique.append(0)
        try:
            y_allCallConnect.append(int(df[df['Date']== day][df['Company']== c]['AllCallConnect']))
        except TypeError:
            y_allCallConnect.append(0)
        try:
            y_allCallUniqeConnect.append(int(df[df['Date']== day][df['Company']== c]['AllCallUniqeConnect']))
        except TypeError:
            y_allCallUniqeConnect.append(0)
    #print( y_allCall , y_allCallUnique , y_allCallConnect  , y_allCallUniqeConnect)
    x_axis = np.array(dates4SQL)
    y_axis_AllCall = np.array(y_allCall)
    y_axis_AllCallUnique = np.array(y_allCallUnique)
    y_axis_AllCallConnect = np.array(y_allCallConnect) 
    y_axis_AllCallUniqueConnect = np.array(y_allCallUniqeConnect) 
        
    plt.figure(figsize=(18, 10 )) 
    plt.xticks(rotation=90)
    plt.plot(x_axis, y_axis_AllCall, linewidth=1.0 , label = "All Calls")
    plt.plot(x_axis, y_axis_AllCallUnique, linewidth=1.0 , label = "All Unique Calls")
    plt.plot(x_axis, y_axis_AllCallConnect, linewidth=1.0 , label = "All Connected Calls")
    plt.plot(x_axis, y_axis_AllCallUniqueConnect, linewidth=1.0 , label = "All Unique Connected Calls")
    
    persian_title = arabic_reshaper.reshape(c + ": تعداد تماس کل، یکتا، موفّق و موفّق یکتا")
    plot_title = get_display(persian_title)
    plt.title(plot_title)
    plt.xlabel("Date")
    plt.ylabel("Number of Calls")
    plt.legend()

    plt.savefig("D:\\CRM\\BigData\\Accidents_Charts\\Output\\" + c + "2.jpg" ,  dpi=600 , orientation='portrait')
    plt.show()
    plt.close()
    
